## Pre-collect all the data in this cell

In [8]:
import json
import os

import requests
from dotenv import load_dotenv
from web3 import Web3

load_dotenv()

ARB_CHAIN_ID = 42161
BALANCER_GAUGE_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-gauges"
BALANCER_GAUGE_CONTROLLER_ADDR = Web3.toChecksumAddress("0xC128468b7Ce63eA702C1f104D55A2566b13D3ABD")
BALANCER_GAUGE_CONTROLLER_ABI = [{"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}],
                                  "outputs": [{"name": "", "type": "uint256"}]},
                                 {"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}, {"name": "time", "type": "uint256"}],
                                  "outputs": [{"name": "", "type": "uint256"}]}]

web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))

# Fetch all voting gauges from github json
voting_gauges_req = requests.get(
    "https://raw.githubusercontent.com/balancer/frontend-v2/master/src/data/voting-gauges.json")
if not voting_gauges_req.ok:
    raise ValueError("Failed to fetch voting gauges")
voting_gauges = voting_gauges_req.json()

# Collect arb gauges
arb_gauges = {}
for gauge in voting_gauges:
    # Only collect gauges for the arb chain and that are not killed
    if int(gauge['network']) == ARB_CHAIN_ID and gauge['isKilled'] is False:
        arb_gauges[gauge['address']] = {
            'gaugeAddress': gauge['address'],
            'pool': gauge['pool']['address'],
            'symbol': gauge['pool']['symbol']
        }

gauge_c_contract = web3.eth.contract(address=BALANCER_GAUGE_CONTROLLER_ADDR, abi=BALANCER_GAUGE_CONTROLLER_ABI)

boost_data = {}
cap_override_data = {}
# Before, load boost data from the file
with open('../data/arbitrumGrantGuageMetadata.json') as f:
    boosties = json.load(f)
    for boost in boosties:
        boost_data[boost['gaugeAddress']] = boost.get('fixedBoost', 1)
        cap_override_data[boost['gaugeAddress']] = boost.get('capOverride', 10)

vote_weights = {}
# Collect gauge voting weights from the gauge controller on chain
for gauge_addr, gauge_data in arb_gauges.items():
    weight = gauge_c_contract.functions.gauge_relative_weight(gauge_addr).call() / 1e18 * 100
    # Boost weight if gauge is boosted, if not boosted, use 1 as multiplier (no boost)
    weight *= boost_data.get(gauge_addr, 1)
    vote_weights[gauge_addr] = weight
    arb_gauges[gauge_addr]['voteWeight'] = weight

## Calculate arbitrum distribution across gauges

In [9]:
from IPython.core.display import HTML
import pandas as pd

ARBITRUM_TO_DISTRIBUTE = 97404
VOTE_CAP_IN_PERCENT = 10  # 10% cap on any single gauge
VOTE_CAPS_IN_PERCENTS = {gauge_addr: cap_override_data.get(gauge_addr, VOTE_CAP_IN_PERCENT) for gauge_addr in
                         arb_gauges.keys()}
# Custom gauge caps taken from boost data
VOTE_CAPS = {gauge_addr: VOTE_CAPS_IN_PERCENTS[gauge_addr] / 100 * ARBITRUM_TO_DISTRIBUTE for gauge_addr in
             arb_gauges.keys()}

# Calculate total weight
total_weight = sum([gauge['voteWeight'] for gauge in arb_gauges.values()])
arb_gauge_distributions = {}
for gauge_addr, gauge_data in arb_gauges.items():
    # Calculate distribution based on vote weight and total weight
    to_distribute = ARBITRUM_TO_DISTRIBUTE * gauge_data['voteWeight'] / total_weight
    # Cap distribution
    to_distribute = to_distribute if to_distribute < VOTE_CAPS[gauge_addr] else VOTE_CAPS[gauge_addr]
    arb_gauge_distributions[gauge_addr] = {
        'gaugeAddress': gauge_addr,
        'pool': gauge_data['pool'],
        'symbol': gauge_data['symbol'],
        'voteWeight': gauge_data['voteWeight'],
        'distribution': to_distribute if to_distribute < VOTE_CAPS[gauge_addr] else VOTE_CAPS[gauge_addr],
        'distributionInPercent': to_distribute / ARBITRUM_TO_DISTRIBUTE * 100,
        'boost': boost_data.get(gauge_addr, 1),
        'capOverride': f"{cap_override_data.get(gauge_addr, VOTE_CAP_IN_PERCENT)}%"
    }


# Spend unspent arb on the gauges that are not capped yet
def recur_distribute_unspend_arb():
    unspent_arb = ARBITRUM_TO_DISTRIBUTE - sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])
    if unspent_arb > 0:
        # Find out total voting weight of uncapped gauges and mark it as 100%:
        total_uncapped_weight = sum(
            [g['voteWeight'] for g in [
                gauge for addr, gauge in arb_gauge_distributions.items() if gauge['distribution'] < VOTE_CAPS[addr]]
             ]
        )
        # Iterate over uncapped gauges and distribute unspent arb proportionally to their voting weight which is total uncapped weight
        for a, uncap_gauge in {addr: gauge for addr, gauge in arb_gauge_distributions.items() if
                               gauge['distribution'] < VOTE_CAPS[addr]}.items():
            # For each loop calculate unspend arb
            unspent_arb = ARBITRUM_TO_DISTRIBUTE - sum(
                [gauge['distribution'] for gauge in arb_gauge_distributions.values()])
            # Don't distribute more than vote cap
            distribution = min(
                uncap_gauge['distribution'] + unspent_arb * uncap_gauge['voteWeight'] / total_uncapped_weight,
                VOTE_CAPS[a])
            uncap_gauge['distribution'] = distribution
            uncap_gauge['distributionInPercent'] = uncap_gauge['distribution'] / ARBITRUM_TO_DISTRIBUTE * 100
    # Call recursively if there is still unspent arb
    if ARBITRUM_TO_DISTRIBUTE - sum([g['distribution'] for g in arb_gauge_distributions.values()]) > 0:
        recur_distribute_unspend_arb()


recur_distribute_unspend_arb()
print(
    f"Unspent arb: {ARBITRUM_TO_DISTRIBUTE - sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])}")
print(f"Arb distributed: {sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])}")
# Remove arb gauges with 0 distribution
arb_gauge_distributions = {addr: gauge for addr, gauge in arb_gauge_distributions.items() if
                           gauge['distribution'] > 0}
arb_gauge_distributions_df = pd.DataFrame.from_dict(arb_gauge_distributions, orient='index')
arb_gauge_distributions_df = arb_gauge_distributions_df.sort_values(by='distributionInPercent', ascending=False)
display(HTML(arb_gauge_distributions_df.to_html(index=False)))

Unspent arb: 0.0
Arb distributed: 97404.0


gaugeAddress,pool,symbol,voteWeight,distribution,distributionInPercent,boost,capOverride
0x9ab40B6e1330Ce70B9e07cD691f281c1539944E6,0xc6EeE8cb7643eC2F05F46d569e9eC8EF8b41b389,bb-a-USD,1.219768,16776.892895,17.224029,1.75,20%
0xb12ADA23eE766bd6b596E2bE556ea2046758b87c,0xBe0f30217BE1e981aDD883848D0773A86d2d2CD4,rETH-bb-a-WETH-BPT,1.020934,14189.392355,14.567566,1.75,20%
0xDf464348c4EC2Bf0e5D6926b9f707c8e02301adf,0x36bf227d6BaC96e2aB1EbB5492ECec69C691943f,B-wstETH-WETH-Stable,0.966584,13792.354499,14.159947,1.75,20%
0x8135d6AbFd42707A87A7b94c5CFA3529f9b432AD,0x32dF62dc3aEd2cD6224193052Ce665DC18165841,RDNT-WETH,0.934983,9740.400000,10.000000,1.50,10%
0x175407b4710b5A1cB67a37C76859F17fb2ff6672,0xc7FA3A3527435720f0e2a4c1378335324dd4F9b3,55auraBal-45wsteth,0.796273,9740.400000,10.000000,1.00,10%
0xa8D974288Fe44ACC329D7d7a179707D27Ec4dd1c,0xc9f52540976385A84BF416903e1Ca3983c539E34,50tBTC-50WETH,0.536536,7535.843868,7.736688,1.00,10%
0x80aed5C5C683FEC86817C98da334DD72424E7297,0x45C4D1376943Ab28802B995aCfFC04903Eb5223f,wstETH-bb-a-WETH-BPT,0.480037,6621.729531,6.798211,1.75,20%
0xacE0D479040231e3c6b17479cFd4444182d521d4,0x567ECfCB22205D279BB8Eed3E066989902bF03D5,DOLA/bbaUSD-BPT,0.373075,5200.385743,5.338986,1.75,10%
0xc4b6cc9A444337b1Cb8cBbDD9de4d983f609C391,0x542F16DA0efB162D20bF4358EfA095B70A100f9E,2BTC,0.266953,3759.065176,3.859251,1.00,10%
0x0EDF6cDd81BC3471C053341B7D8Dfd1Cb367AD93,0xb3028Ca124B80CFE6E9CA57B70eF2F0CCC41eBd4,50MAGIC-50USDC,0.209722,3000.257779,3.080220,1.00,10%


## Export to json

In [10]:
# Export to json
with open('../data/arbitrumGaugeDistribution.json', 'w') as f:
    json.dump(arb_gauge_distributions, f, indent=4)